In [ ]:
from progressivis_nb_widgets import *

In [ ]:
"""
Clustering datasets may be found at
https://cs.joensuu.fi/sipu/datasets/
"""
from progressivis import Scheduler, Every#, log_level
from progressivis.cluster import MBKMeans
from progressivis.io import CSVLoader
from progressivis.vis import MCScatterPlot
from progressivis.datasets import get_dataset
from progressivis.stats import RandomTable
from progressivis.core.utils import gather_and_run
from progressivis.utils.psdict import PsDict
import numpy as np
try:
    s = scheduler
except NameError:
    s = Scheduler()
    #log_level(package="progressivis.cluster")

data = CSVLoader(get_dataset('cluster:s1'),sep='\\s+',skipinitialspace=True,header=None,index_col=False,scheduler=s)
mbkmeans = MBKMeans(columns=['_0', '_1'], n_clusters=15, batch_size=100, is_input=False, scheduler=s)
sp = MCScatterPlot(scheduler=s, classes=[('Scatterplot', '_0', '_1', mbkmeans)])
sp.create_dependent_modules(data,'result')
sp['Scatterplot'].min_value.result = PsDict({'_0': -np.inf, '_1': -np.inf})
sp['Scatterplot'].max_value.result = PsDict({'_0': np.inf, '_1': np.inf})
mbkmeans.create_dependent_modules(sp['Scatterplot'].range_query_2d)

sp.move_point = mbkmeans.moved_center # for input management
# Create the dashboard object

psboard = PsBoard(s, order='desc')
# Visualisations require registration :
# 1) When widget provides the link_module() method do :
psboard.register_visualisation(Scatterplot(), sp)
# 2) When widget is generic or for overloading the link_module() method do:
# psboard.register_visualisation(FooWidget(), bar_module, glue=glue_func)
# NB: glue_func(widget, module)-> [coroutine] must be provided
# Start application :
gather_and_run(s.start(), *psboard.coroutines)

#### Click  _&#x1f441; mc_scatter_plot_1_ below to display the scatterplot:

In [ ]:
display(psboard)